In [1]:
from ultralytics import YOLO
import cv2

model = YOLO("yolo8x-segment-barcodes-bboxed.pt")
# accepts all formats - image/dir/Path/URL/video/PIL/ndarray. 0 for webcam
# results = model.predict(source="0")
# results = model.predict(source="folder", show=True) # Display preds. Accepts all YOLO predict arguments

# from ndarray
# img = cv2.imread("C:/Users/Artem/bboxed_barcodes_dataset/Training/images/msg886302187-765211.jpg")
# results = model.predict(source=img, save=True, save_txt=True)  # save predictions as labels

In [2]:
# results = model.val(data="bboxed_barcodes_dataset/data.yaml", save=True, save_txt=True)
results = model.predict(source="bboxed_barcodes_dataset/Validation/images/*", save=True, save_txt=True)


image 1/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-765200.jpg: 256x256 1 2d, 20.0ms
image 2/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-765201.jpg: 256x256 1 2d, 16.0ms
image 3/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-7652010.jpg: 160x256 1 2d, 190.0ms
image 4/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-7652011.jpg: 192x256 1 2d, 200.0ms
image 5/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-765202.jpg: 256x224 1 2d, 219.9ms
image 6/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-765203.jpg: 224x256 1 2d, 203.7ms
image 7/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-765204.jpg: 256x256 1 2d, 19.0ms
image 8/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-765205.jpg: 224x256 1 2d, 25.0ms
image 9/49 C:\Users\Artem\bboxed_barcodes_dataset\Validation\images\msg886302187-

In [3]:
import os
import numpy as np
from copy import deepcopy

ground_truth_masks = []
predicted_masks = []

path_to_predicted_labels = "C:/Users/Artem/runs/segment/predict/labels/"
path_to_gt_labels = "C:/Users/Artem/bboxed_barcodes_dataset/Validation/labels/"
path_to_images = "C:/Users/Artem/bboxed_barcodes_dataset/Validation/images/"

images_names = os.listdir(path_to_images)

for name in images_names:
    name = name[:-4]
    img = cv2.imread(path_to_images+name+".jpg")
    height, width, _ = img.shape
    pred_mask = np.zeros((height, width, 1), dtype=np.uint8)
    gt_mask = np.zeros((height, width, 1), dtype=np.uint8)
    predicted_points = []
    gt_points = []

    with open(path_to_predicted_labels+name+".txt", 'r') as fin:
        data = fin.readline().split(' ')[1:]
        for i in range(int(len(data)/2)):
            predicted_points.append([int(float(data[i*2])*width), int(float(data[i*2+1])*height)])

    with open(path_to_gt_labels+name+".txt", 'r') as fin:
        data = fin.readline().split(' ')[1:]
        for i in range(int(len(data)/2)):
            gt_points.append([int(float(data[i*2])*width), int(float(data[i*2+1])*height)])

    predicted_points = np.array(predicted_points)
    gt_points = np.array(gt_points)

    predicted_masks.append(cv2.fillPoly(pred_mask, pts=[predicted_points], color=(255)))
    ground_truth_masks.append(cv2.fillPoly(gt_mask, pts=[gt_points], color=(255)))

In [4]:
def calculate_iou(pred_mask, gt_mask):
    intersection = np.logical_and(pred_mask, gt_mask).sum()
    union = np.logical_or(pred_mask, gt_mask).sum()
    iou = intersection / union if union > 0 else 0
    return iou

In [8]:
ious = [calculate_iou(predicted_masks[i], ground_truth_masks[i]) for i in range(len(predicted_masks))]
print(f"mean iou: {np.mean(ious)}")
ious

mean iou: 0.8285856911690201


[0.9250814332247557,
 0.9221209610604806,
 0.9204912734324499,
 0.923941798941799,
 0.9477981918926801,
 0.9076376554174067,
 0.9125441696113075,
 0.9337717238139972,
 0.9120567375886525,
 0.924219910846954,
 0.9219814241486068,
 0.910941475826972,
 0.9082900384007229,
 0.9491853524762058,
 0.9333910034602076,
 0.8346028291621328,
 0.8189845474613686,
 0.9058734939759037,
 0.8701794578083238,
 0.8029948525970987,
 0.0,
 0.19558529872092395,
 0.9356563056820432,
 0.9158706833210875,
 0.0,
 0.8268824771287826,
 0.9112808460634547,
 0.8120617110799438,
 0.7713801862828111,
 0.285337552742616,
 0.9344853462292075,
 0.9490959768217992,
 0.9383006535947712,
 0.9347374129982826,
 0.9398009950248756,
 0.8697658519911127,
 0.8851774530271399,
 0.8817991631799164,
 0.8861985472154964,
 0.7997039960532807,
 0.8866363260239967,
 0.9261207112446782,
 0.7919732441471572,
 0.8482293423271501,
 0.942756183745583,
 0.9219242902208202,
 0.9089108910891089,
 0.7070443533358182,
 0.9078947368421053]